In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed,SimpleRNN,Dropout,GRU
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [2]:
df = pd.read_excel("D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_train.xlsx")
df_truth = pd.read_excel("D:\\Projects\\BDA\\aircraft_reliability\\data\\PM_truth.xlsx")


In [3]:
df_merged = pd.merge(df, df_truth, on='id')
df_merged

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,more
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,98
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,98
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,98
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,98
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20426,99,181,-0.0015,-0.0001,100,518.67,643.72,1600.39,1428.03,14.62,...,2388.29,8123.55,8.4885,0.03,396,2388,100,38.50,23.0425,20
20427,99,182,-0.0027,-0.0003,100,518.67,643.52,1605.33,1430.32,14.62,...,2388.27,8130.99,8.5124,0.03,393,2388,100,38.39,22.9674,20
20428,99,183,-0.0031,-0.0003,100,518.67,643.96,1606.95,1427.90,14.62,...,2388.31,8126.90,8.5374,0.03,395,2388,100,38.57,23.1440,20
20429,99,184,-0.0010,-0.0001,100,518.67,644.10,1600.20,1436.54,14.62,...,2388.33,8125.66,8.5592,0.03,395,2388,100,38.45,23.0478,20


In [4]:
# Step 1: Get the maximum cycle for each engine
max_cycle_per_engine = df.groupby('id')['cycle'].max().reset_index()
max_cycle_per_engine.columns = ['id', 'max_cycle']

# Step 2: Merge the maximum cycle with the df_truth to get the actual failure cycle
df_merged = pd.merge(max_cycle_per_engine, df_truth, on='id')

# Step 3: Calculate the actual failure cycle (when engine will fail)
df_merged['failure_cycle'] = df_merged['max_cycle'] + df_merged['more']

# Step 4: Merge this back with the main DataFrame to compute remaining cycles
df = pd.merge(df, df_merged[['id', 'failure_cycle']], on='id')

# Step 5: Calculate remaining cycles for each row by subtracting the current cycle from the failure cycle
df['remaining_cycles'] = df['failure_cycle'] - df['cycle']
df = df.drop('failure_cycle',axis=1)

In [5]:
df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,remaining_cycles
0,1,1,-0.0007,-0.0004,100,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100,39.06,23.4190,289
1,1,2,0.0019,-0.0003,100,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100,39.00,23.4236,288
2,1,3,-0.0043,0.0003,100,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100,38.95,23.3442,287
3,1,4,0.0007,0.0000,100,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100,38.88,23.3739,286
4,1,5,-0.0019,-0.0002,100,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100,38.90,23.4044,285


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20431 entries, 0 to 20430
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                20431 non-null  int64  
 1   cycle             20431 non-null  int64  
 2   setting1          20431 non-null  float64
 3   setting2          20431 non-null  float64
 4   setting3          20431 non-null  int64  
 5   s1                20431 non-null  float64
 6   s2                20431 non-null  float64
 7   s3                20431 non-null  float64
 8   s4                20431 non-null  float64
 9   s5                20431 non-null  float64
 10  s6                20431 non-null  float64
 11  s7                20431 non-null  float64
 12  s8                20431 non-null  float64
 13  s9                20431 non-null  float64
 14  s10               20431 non-null  float64
 15  s11               20431 non-null  float64
 16  s12               20431 non-null  float6

In [7]:
def create_sequences_3(df, window_size=30, test_size=0.1, val_size=0.1):
    """
    Generates scaled sequences and splits into training, validation, and testing sets.
    
    Parameters:
    df (pd.DataFrame): The input dataframe with engine data.
    window_size (int): Number of time steps in each sequence.
    test_size (float): Fraction of data to reserve for testing.
    val_size (float): Fraction of data to reserve for validation.

    Returns:
    X_train, X_val, X_test, y_train, y_val, y_test, feature_scaler, target_scaler (np.ndarray): 
    Arrays of train/val/test sequences and targets, feature and target scalers.
    """
    features = [col for col in df.columns if col not in ['id', 'cycle', 'remaining_cycles']]
    target_column = 'remaining_cycles'
    
    # Initialize scalers
    feature_scaler = StandardScaler()
    target_scaler = StandardScaler()
    
    # Lists to collect sequences and targets
    X_sequences = []
    y_targets = []
    
    # Group by engine ID and create sliding windows before splitting
    for engine_id, engine_data in df.groupby('id'):
        engine_data = engine_data.sort_values(by='cycle')  # Sort by cycle to keep temporal order
        
        # Create sliding windows
        for i in range(len(engine_data) - window_size):
            X_sequence = engine_data[features].iloc[i:i + window_size].values
            y_target = engine_data[target_column].iloc[i + window_size - 1]  # Target is RUL of last cycle in window
            
            X_sequences.append(X_sequence)
            y_targets.append(y_target)
    
    # Convert to numpy arrays
    X_sequences = np.array(X_sequences)
    y_targets = np.array(y_targets)
    
    # Split data into training+validation and testing
    X_train_val, X_test, y_train_val, y_test = train_test_split(X_sequences, y_targets, test_size=test_size, random_state=42)
    
    # Further split the training+validation data into training and validation
    val_ratio_adjusted = val_size / (1 - test_size)  # Adjust validation size proportionally
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=val_ratio_adjusted, random_state=42)
    
    # Fit the scalers on the training data
    feature_scaler.fit(X_train.reshape(-1, X_train.shape[2]))  # Reshape for fitting scaler
    target_scaler.fit(y_train.reshape(-1, 1))  # Reshape target for fitting scaler
    
    # Apply the scalers to the data
    X_train = feature_scaler.transform(X_train.reshape(-1, X_train.shape[2])).reshape(X_train.shape)  # Reshape back after scaling
    X_val = feature_scaler.transform(X_val.reshape(-1, X_val.shape[2])).reshape(X_val.shape)
    X_test = feature_scaler.transform(X_test.reshape(-1, X_test.shape[2])).reshape(X_test.shape)
    
    y_train = target_scaler.transform(y_train.reshape(-1, 1))
    y_val = target_scaler.transform(y_val.reshape(-1, 1))
    y_test = target_scaler.transform(y_test.reshape(-1, 1))
    
    return X_train, X_val, X_test, y_train, y_val, y_test, feature_scaler, target_scaler


In [8]:
def plot_loss(history, title,lim=(None,None)):
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.ylim(lim)
    plt.grid(True)
    plt.show()


In [9]:
# Adjusted evaluation function
def evaluate_model(model, X_test, y_test,type,scaler):
    predictions = model.predict(X_test)
    if predictions.shape[0] != y_test.shape[0]:
        # Trimming to the minimum size for consistency
        min_len = min(predictions.shape[0], y_test.shape[0])
        predictions = predictions[:min_len]
        y_test = y_test[:min_len]
    rmse = np.sqrt(mean_squared_error(scaler.inverse_transform(y_test.reshape(-1,1)), scaler.inverse_transform(predictions.reshape(-1,1))))
    mae = mean_absolute_error(scaler.inverse_transform(y_test.reshape(-1,1)), scaler.inverse_transform(predictions.reshape(-1,1)))
    print(f"{type} Model Evaluation - RMSE: {rmse:.2f}, MAE: {mae:.2f}")
    return predictions, rmse, mae

In [10]:
window_size_1=5
window_size_2=10
window_size_3=20
window_size_4=30
window_size_5=40
window_size_6=50

num_features=24

In [11]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_1, test_size=0.2)

model_1 = Sequential([
    GRU(64, return_sequences=True, input_shape=(window_size_1, num_features)),
    Dropout(0.3),
    GRU(32, return_sequences=True),
    Dropout(0.2),
    GRU(16, return_sequences=False),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])
model_1.compile(optimizer=Adam(learning_rate=0.001),loss='mae',metrics=['mse'])
# Train the model
history_1 = model_1.fit(X_train,y_train,
                    validation_data=(X_val,y_val),
                    epochs=200, batch_size=16,
                    callbacks=tf.keras.callbacks.EarlyStopping(patience=30,monitor='val_loss',restore_best_weights=True))

Epoch 1/200
873/873 [==============================] - 35s 25ms/step - loss: 0.6105 - mse: 0.5824 - val_loss: 0.5824 - val_mse: 0.5286
Epoch 2/200
873/873 [==============================] - 21s 24ms/step - loss: 0.6004 - mse: 0.5672 - val_loss: 0.5816 - val_mse: 0.5292
Epoch 3/200
873/873 [==============================] - 22s 25ms/step - loss: 0.5950 - mse: 0.5593 - val_loss: 0.5741 - val_mse: 0.5152
Epoch 4/200
873/873 [==============================] - 19s 21ms/step - loss: 0.5922 - mse: 0.5554 - val_loss: 0.5752 - val_mse: 0.5201
Epoch 5/200
873/873 [==============================] - 21s 24ms/step - loss: 0.5870 - mse: 0.5489 - val_loss: 0.5761 - val_mse: 0.5214
Epoch 6/200
873/873 [==============================] - 20s 23ms/step - loss: 0.5856 - mse: 0.5472 - val_loss: 0.5675 - val_mse: 0.5101
Epoch 7/200
873/873 [==============================] - 21s 24ms/step - loss: 0.5812 - mse: 0.5431 - val_loss: 0.5657 - val_mse: 0.5098
Epoch 8/200
873/873 [==============================] - 

In [12]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_2, test_size=0.2)

model_2 = Sequential([
    GRU(64, return_sequences=True, input_shape=(window_size_2, num_features)),
    Dropout(0.3),
    GRU(32, return_sequences=True),
    Dropout(0.2),
    GRU(16, return_sequences=False),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])
model_2.compile(optimizer=Adam(learning_rate=0.001),loss='mae',metrics=['mse'])
# Train the model
history_2 = model_2.fit(X_train,y_train,
                    validation_data=(X_val,y_val),
                    epochs=200, batch_size=16,
                    callbacks=tf.keras.callbacks.EarlyStopping(patience=30,monitor='val_loss',restore_best_weights=True))

Epoch 1/200
851/851 [==============================] - 38s 31ms/step - loss: 0.6153 - mse: 0.5915 - val_loss: 0.5893 - val_mse: 0.5579
Epoch 2/200
851/851 [==============================] - 23s 27ms/step - loss: 0.5988 - mse: 0.5675 - val_loss: 0.5900 - val_mse: 0.5634
Epoch 3/200
851/851 [==============================] - 22s 26ms/step - loss: 0.5872 - mse: 0.5536 - val_loss: 0.5784 - val_mse: 0.5448
Epoch 4/200
851/851 [==============================] - 23s 27ms/step - loss: 0.5729 - mse: 0.5340 - val_loss: 0.5609 - val_mse: 0.5207
Epoch 5/200
851/851 [==============================] - 22s 26ms/step - loss: 0.5593 - mse: 0.5201 - val_loss: 0.5700 - val_mse: 0.5361
Epoch 6/200
851/851 [==============================] - 22s 26ms/step - loss: 0.5376 - mse: 0.4891 - val_loss: 0.5528 - val_mse: 0.5355
Epoch 7/200
851/851 [==============================] - 21s 24ms/step - loss: 0.5108 - mse: 0.4515 - val_loss: 0.5157 - val_mse: 0.4594
Epoch 8/200
851/851 [==============================] - 

In [13]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_3, test_size=0.2)

model_3 = Sequential([
    GRU(64, return_sequences=True, input_shape=(window_size_3, num_features)),
    Dropout(0.3),
    GRU(32, return_sequences=True),
    Dropout(0.2),
    GRU(16, return_sequences=False),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])
model_3.compile(optimizer=Adam(learning_rate=0.001),loss='mae',metrics=['mse'])
# Train the model
history_3 = model_3.fit(X_train,y_train,
                    validation_data=(X_val,y_val),
                    epochs=200, batch_size=16,
                    callbacks=tf.keras.callbacks.EarlyStopping(patience=30,monitor='val_loss',restore_best_weights=True))

Epoch 1/200
808/808 [==============================] - 35s 33ms/step - loss: 0.6176 - mse: 0.5948 - val_loss: 0.6119 - val_mse: 0.5641
Epoch 2/200
808/808 [==============================] - 25s 30ms/step - loss: 0.5953 - mse: 0.5603 - val_loss: 0.5704 - val_mse: 0.5290
Epoch 3/200
808/808 [==============================] - 24s 30ms/step - loss: 0.5667 - mse: 0.5205 - val_loss: 0.5713 - val_mse: 0.5561
Epoch 4/200
808/808 [==============================] - 25s 31ms/step - loss: 0.5283 - mse: 0.4697 - val_loss: 0.5033 - val_mse: 0.4366
Epoch 5/200
808/808 [==============================] - 26s 32ms/step - loss: 0.4785 - mse: 0.4010 - val_loss: 0.4594 - val_mse: 0.3667
Epoch 6/200
808/808 [==============================] - 25s 31ms/step - loss: 0.4286 - mse: 0.3340 - val_loss: 0.4351 - val_mse: 0.3240
Epoch 7/200
808/808 [==============================] - 25s 31ms/step - loss: 0.3810 - mse: 0.2727 - val_loss: 0.3633 - val_mse: 0.2673
Epoch 8/200
808/808 [==============================] - 

In [14]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_4, test_size=0.2)

model_4 = Sequential([
    GRU(64, return_sequences=True, input_shape=(window_size_4, num_features)),
    Dropout(0.3),
    GRU(32, return_sequences=True),
    Dropout(0.2),
    GRU(16, return_sequences=False),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])
model_4.compile(optimizer=Adam(learning_rate=0.001),loss='mae',metrics=['mse'])
# Train the model
history_4 = model_4.fit(X_train,y_train,
                    validation_data=(X_val,y_val),
                    epochs=200, batch_size=16,
                    callbacks=tf.keras.callbacks.EarlyStopping(patience=30,monitor='val_loss',restore_best_weights=True))

Epoch 1/200
764/764 [==============================] - 46s 48ms/step - loss: 0.6221 - mse: 0.5962 - val_loss: 0.5998 - val_mse: 0.5573
Epoch 2/200
764/764 [==============================] - 34s 45ms/step - loss: 0.6009 - mse: 0.5674 - val_loss: 0.5748 - val_mse: 0.5235
Epoch 3/200
764/764 [==============================] - 33s 43ms/step - loss: 0.5518 - mse: 0.5000 - val_loss: 0.5019 - val_mse: 0.4119
Epoch 4/200
764/764 [==============================] - 34s 44ms/step - loss: 0.4821 - mse: 0.3993 - val_loss: 0.4293 - val_mse: 0.3284
Epoch 5/200
764/764 [==============================] - 34s 45ms/step - loss: 0.4010 - mse: 0.2843 - val_loss: 0.3589 - val_mse: 0.2241
Epoch 6/200
764/764 [==============================] - 34s 44ms/step - loss: 0.3305 - mse: 0.1991 - val_loss: 0.3131 - val_mse: 0.1728
Epoch 7/200
764/764 [==============================] - 33s 43ms/step - loss: 0.2803 - mse: 0.1452 - val_loss: 0.2538 - val_mse: 0.1164
Epoch 8/200
764/764 [==============================] - 

In [15]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_5, test_size=0.2)

model_5 = Sequential([
    GRU(64, return_sequences=True, input_shape=(window_size_5, num_features)),
    Dropout(0.3),
    GRU(32, return_sequences=True),
    Dropout(0.2),
    GRU(16, return_sequences=False),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])
model_5.compile(optimizer=Adam(learning_rate=0.001),loss='mae',metrics=['mse'])
# Train the model
history_5 = model_5.fit(X_train,y_train,
                    validation_data=(X_val,y_val),
                    epochs=200, batch_size=16,
                    callbacks=tf.keras.callbacks.EarlyStopping(patience=30,monitor='val_loss',restore_best_weights=True))

c:\Users\anush\.conda\envs\mlEnv\lib\site-packages\sklearn\preprocessing\_data.py:979: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask


Epoch 1/200
721/721 [==============================] - 49s 57ms/step - loss: 0.6252 - mse: 0.6081 - val_loss: 0.6076 - val_mse: 0.5716
Epoch 2/200
721/721 [==============================] - 39s 54ms/step - loss: 0.5958 - mse: 0.5628 - val_loss: 0.5348 - val_mse: 0.4833
Epoch 3/200
721/721 [==============================] - 39s 54ms/step - loss: 0.5268 - mse: 0.4617 - val_loss: 0.5430 - val_mse: 0.4685
Epoch 4/200
721/721 [==============================] - 39s 54ms/step - loss: 0.4326 - mse: 0.3252 - val_loss: 0.3579 - val_mse: 0.2358
Epoch 5/200
721/721 [==============================] - 39s 54ms/step - loss: 0.3516 - mse: 0.2240 - val_loss: 0.3279 - val_mse: 0.1805
Epoch 6/200
721/721 [==============================] - 39s 55ms/step - loss: 0.2902 - mse: 0.1563 - val_loss: 0.2565 - val_mse: 0.1334
Epoch 7/200
721/721 [==============================] - 39s 54ms/step - loss: 0.2352 - mse: 0.1050 - val_loss: 0.2095 - val_mse: 0.0806
Epoch 8/200
721/721 [==============================] - 

In [16]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_6, test_size=0.2)

model_6 = Sequential([
    GRU(64, return_sequences=True, input_shape=(window_size_6, num_features)),
    Dropout(0.3),
    GRU(32, return_sequences=True),
    Dropout(0.2),
    GRU(16, return_sequences=False),
    Dropout(0.1),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])
model_6.compile(optimizer=Adam(learning_rate=0.001),loss='mae',metrics=['mse'])
# Train the model
history_6 = model_6.fit(X_train,y_train,
                   validation_data=(X_val,y_val),
                    epochs=200, batch_size=16,
                    callbacks=tf.keras.callbacks.EarlyStopping(patience=30,monitor='val_loss',restore_best_weights=True))

Epoch 1/200
678/678 [==============================] - 55s 70ms/step - loss: 0.6351 - mse: 0.6253 - val_loss: 0.6181 - val_mse: 0.6110
Epoch 2/200
678/678 [==============================] - 46s 67ms/step - loss: 0.6066 - mse: 0.5840 - val_loss: 0.5776 - val_mse: 0.5418
Epoch 3/200
678/678 [==============================] - 46s 69ms/step - loss: 0.5611 - mse: 0.5219 - val_loss: 0.4953 - val_mse: 0.4278
Epoch 4/200
678/678 [==============================] - 45s 67ms/step - loss: 0.4688 - mse: 0.3911 - val_loss: 0.3992 - val_mse: 0.2964
Epoch 5/200
678/678 [==============================] - 46s 67ms/step - loss: 0.3638 - mse: 0.2522 - val_loss: 0.2933 - val_mse: 0.1680
Epoch 6/200
678/678 [==============================] - 45s 67ms/step - loss: 0.2883 - mse: 0.1635 - val_loss: 0.2268 - val_mse: 0.1037
Epoch 7/200
678/678 [==============================] - 45s 67ms/step - loss: 0.2269 - mse: 0.1006 - val_loss: 0.1865 - val_mse: 0.0705
Epoch 8/200
678/678 [==============================] - 

In [17]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_1, test_size=0.2)
evaluate_model(model_1, X_test,y_test,f'window_len {window_size_1} model',yscaler)


125/125 [==============================] - 2s 5ms/step
window_len 5 model Model Evaluation - RMSE: 52.13, MAE: 39.62


(array([[-1.3000412 ],
        [ 1.1116015 ],
        [ 0.6796353 ],
        ...,
        [-1.3276074 ],
        [ 0.21961956],
        [-1.2511168 ]], dtype=float32),
 52.13059984094697,
 39.61638511267445)

In [18]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_2, test_size=0.2)
evaluate_model(model_2, X_test,y_test,f'window_len {window_size_2} model',yscaler)


122/122 [==============================] - 2s 7ms/step
window_len 10 model Model Evaluation - RMSE: 23.73, MAE: 16.49


(array([[ 0.82974136],
        [-0.9466406 ],
        [-1.7531383 ],
        ...,
        [ 1.1574082 ],
        [ 0.9325977 ],
        [ 0.11034414]], dtype=float32),
 23.73402431678229,
 16.488184993878278)

In [19]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_3, test_size=0.2)
evaluate_model(model_3, X_test,y_test,f'window_len {window_size_3} model',yscaler)


116/116 [==============================] - 3s 11ms/step
window_len 20 model Model Evaluation - RMSE: 9.00, MAE: 6.67


(array([[-0.2991754 ],
        [-0.5453476 ],
        [-0.08779128],
        ...,
        [ 0.09589133],
        [-0.95557123],
        [-0.2911966 ]], dtype=float32),
 8.996871776098274,
 6.6653618534804355)

In [20]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_4, test_size=0.2)
evaluate_model(model_4, X_test,y_test,f'window_len {window_size_4} model',yscaler)


110/110 [==============================] - 3s 17ms/step
window_len 30 model Model Evaluation - RMSE: 6.96, MAE: 5.09


(array([[-0.53057575],
        [-1.279528  ],
        [ 0.7114718 ],
        ...,
        [-0.8680728 ],
        [ 0.84354126],
        [-1.7484484 ]], dtype=float32),
 6.962075660399104,
 5.091556260758473)

In [21]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_5, test_size=0.2)
evaluate_model(model_5, X_test,y_test,f'window_len {window_size_5} model',yscaler)


c:\Users\anush\.conda\envs\mlEnv\lib\site-packages\sklearn\preprocessing\_data.py:979: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(self.var_), copy=False, constant_mask=constant_mask


103/103 [==============================] - 6s 24ms/step
window_len 40 model Model Evaluation - RMSE: 5.59, MAE: 4.33


(array([[-0.8582148],
        [-0.5460262],
        [-0.7912904],
        ...,
        [-0.7975193],
        [-1.2992631],
        [-1.8220927]], dtype=float32),
 5.591231034703361,
 4.330707889840527)

In [22]:
X_train, X_val, X_test, y_train, y_val, y_test, scaler,yscaler= create_sequences_3(df, window_size=window_size_6, test_size=0.2)
evaluate_model(model_6, X_test,y_test,f'window_len {window_size_6} model',yscaler)


97/97 [==============================] - 4s 28ms/step
window_len 50 model Model Evaluation - RMSE: 5.86, MAE: 4.31


(array([[ 0.18080445],
        [ 1.0308777 ],
        [ 0.41731155],
        ...,
        [ 1.351302  ],
        [-0.55051386],
        [-0.58213365]], dtype=float32),
 5.858551991722847,
 4.310656302738159)